In [ ]:
# TODO:
# - Geocode
# - Average income
# - Population density
# - Competitors
# - Cost of living impact
# - COVID impact?
# - Footfall?

### Clean and add UUIDs

In [ ]:
import uuid
import pandas as pd

aldi_df = pd.read_csv("../data/scraped_stores.csv")
aldi_df = aldi_df.iloc[:, 1:]
aldi_df["uuid"] = [str(uuid.uuid4()) for _ in range(aldi_df.shape[0])]

## Geocode store addresses

In [ ]:
import os
import requests
import geopandas as gpd
from dotenv import load_dotenv
from shapely.geometry import Point
from time import sleep

load_dotenv("../.env")
tomtom_api_key = os.environ.get("TOMTOM_API_KEY")

base_url = "https://api.tomtom.com/search/2/geocode/"

def geocode(address: str) -> Point:
    sleep(0.5)
    print(f"Geocoding {address}")
    res = requests.get(f"{base_url}ALDI Supermarket, {address}.json?key={tomtom_api_key}&countrySet=GB")

    if res.status_code != 200:
        print("FAILED: Bad status code")
        return None
    if res.json().get("summary").get("totalResults") <= 0:
        print("FAILED: No results")
        return None

    results = res.json().get("results")
    lat = results[0].get("position", {}).get("lat")
    lon = results[0].get("position", {}).get("lon")
    return Point(lon, lat)

aldi_df["geometry"] = aldi_df["full_address"].apply(geocode)
aldi_gdf = gpd.GeoDataFrame(aldi_df, crs="EPSG:4326")
aldi_gdf.to_file("../data/geocoded_stores.geojson", driver="GeoJSON")